## Speed for three Trajectory classes in `pytraj`

What are they?
    * TrajReadOnly: a thin wrapper for cpptraj Trajin classes. Data will only be loaded if iterating it
    * FrameArray: a wrapper of C++ vector of Frame object. Data will be fully loaded to memory. This class was originally designed to hold in-memory Frames for TrajReadOnly and for faster iterating
    * api.Trajectory: new class using numpy
        * why this class: it's very fast to iterate FrameArray but it's a bit slow to convert to numpy array. I created this class as an interface to numpy/scipy/mdtraj ....
        * Note: FrameArray might be deprecated, depending on how I (we) really need. For technical diffculty in manage lifetime of C++ vector in Python, I can not subclass FrameArray in Python level. 

In [1]:
from pytraj.__version__ import __version__ as p_version
import numpy as np
import mdtraj as md
print (p_version)

0.1.2.dev4


## loading data

In [2]:
# data is stored in local disk, generated from TIP3P REMD, netcdf, 200 Mb, 17443 atoms, 1000 frames

from pytraj import io
import mdtraj as md

top_name = "../tests/data/nogit/remd/myparm.parm7"
filename = "../tests/data/nogit/remd/remd.x.000"
file_and_top_names = (filename, top_name)

In [3]:
# load classes
from pytraj import FrameArray
from pytraj.api import Trajectory
from pytraj import TrajReadOnly

In [4]:
%time fa = FrameArray(*file_and_top_names)
%time traj = Trajectory(*file_and_top_names)
%time trajread = TrajReadOnly(*file_and_top_names)

CPU times: user 2.43 s, sys: 839 ms, total: 3.26 s
Wall time: 4.18 s
CPU times: user 2.24 s, sys: 428 ms, total: 2.66 s
Wall time: 2.68 s
CPU times: user 207 ms, sys: 12 ms, total: 219 ms
Wall time: 219 ms


In [5]:
# reload since we used "%timeit".

fa = FrameArray(*file_and_top_names)
traj = Trajectory(*file_and_top_names)
trajread = TrajReadOnly(*file_and_top_names)

## iterate frames

In [6]:
%timeit for frame in fa: pass
%timeit for frame in traj: pass
%timeit for frame in trajread: pass

# iterate FrameArray is extremely fast since it is a wrapper of C++ vector class

1000 loops, best of 3: 844 µs per loop
1 loops, best of 3: 248 ms per loop
1 loops, best of 3: 1.02 s per loop


In [7]:
print ("iter FrameArray")
%timeit for frame in fa: pass
print ("iter FrameArray and create numpy memoryview for Frame coords: use `asarray`")
%timeit for frame in fa: xyz = np.asarray(frame[:])
print ("iter FrameArray and create numpy array without memoryview for Frame coords: use `array`")
%timeit for frame in fa: xyz = np.array(frame[:])

print ("")
print ("iter Traj (numpy version)")
%timeit for frame in traj: pass
print ("iter Traj (numpy) and create numpy memoryview for Frame coords")
%timeit for frame in traj: xyz = np.asarray(frame[:])

print ("")
print ("iter TrajReadOnly")
%timeit for frame in trajread: pass
print ("iter TrajReadOnly and create numpy memoryview for Frame coords")
%timeit for frame in trajread: xyz = np.asarray(frame[:])

# iterate FrameArray is extremely fast since it is a wrapper of C++ vector class

iter FrameArray
1000 loops, best of 3: 836 µs per loop
iter FrameArray and create numpy memoryview for Frame coords: use `asarray`
10 loops, best of 3: 31.4 ms per loop
iter FrameArray and create numpy array without memoryview for Frame coords: use `array`
1 loops, best of 3: 218 ms per loop

iter Traj (numpy version)
1 loops, best of 3: 247 ms per loop
iter Traj (numpy) and create numpy memoryview for Frame coords
1 loops, best of 3: 289 ms per loop

iter TrajReadOnly
1 loops, best of 3: 1.02 s per loop
iter TrajReadOnly and create numpy memoryview for Frame coords
1 loops, best of 3: 1.1 s per loop


## frame_iter

In [8]:
start, stop, stride = 10, 999, 100

%timeit for frame in fa.frame_iter(start, stop, stride): pass
%timeit for frame in traj.frame_iter(start, stop, stride): pass
%timeit for frame in trajread.frame_iter(start, stop, stride): pass

# we need to redesign frame_iter of Trajectory

1000 loops, best of 3: 1.38 ms per loop
100 loops, best of 3: 2.98 ms per loop
100 loops, best of 3: 15.4 ms per loop


## convert to numpy array by calling `xyz` attribute

In [9]:
%timeit fa.xyz
%timeit traj.xyz
%timeit trajread.xyz

# traj.xyz is numpy array, so it tooks only 115 ns to call (vs. 504 ms for FrameArray and 2.6 s for TrajReadOnly).
# However iterating Trajectory (numpy) is much slower.
# the best thing is to move numpy to Cython level so we can combine fast iterating in Cython + fast vectorizing in numpy

1 loops, best of 3: 517 ms per loop
The slowest run took 34.66 times longer than the fastest. This could mean that an intermediate result is being cached 
10000000 loops, best of 3: 115 ns per loop
1 loops, best of 3: 2.79 s per loop


## slicing

In [10]:
import mdtraj as md

top_name = "../tests/data/nogit/remd/myparm.parm7"
filename = "../tests/data/nogit/remd/remd.x.000"
m_top = md.load_prmtop(top_name)
m_traj = md.load_netcdf(filename, top=m_top)

In [16]:
## slicing

s = slice(0, 1000, 5)
print("FrameArray")
%timeit fa[s]

print ("api.Trajectory")
%timeit traj[s]

print ("TrajReadOnly")
%timeit trajread[s]

print ("mdtraj")
%timeit m_traj[s]

# make sure we sliced the trajs correctly
print (fa[s])
print (traj[s])
print (trajread[s])
print (m_traj[s])

FrameArray
1 loops, best of 3: 315 ms per loop
api.Trajectory
The slowest run took 4.41 times longer than the fastest. This could mean that an intermediate result is being cached 
100000 loops, best of 3: 17.9 µs per loop
TrajReadOnly
1 loops, best of 3: 486 ms per loop
mdtraj
10 loops, best of 3: 189 ms per loop
<FrameArray with 200 frames, 17443 atoms/frame>
           
Trajectory with 200 frames, 17443 atoms
<FrameArray with 200 frames, 17443 atoms/frame>
           
<mdtraj.Trajectory with 200 frames, 17443 atoms, 5666 residues, and unitcells>


## asignment

In [12]:
xyz = traj.xyz.copy()

print("FrameArray")
%timeit for i, frame in enumerate(fa): frame[:] = xyz[i]

print ("api.Trajectory: frame iteration")
%timeit for i, frame in enumerate(traj): frame[:] = xyz[i]
    
print ("api.Trajectory: numpy")
%timeit traj.xyz[:] = xyz

print ("TrajReadOnly") # note: just for testing, xyz assigment does not change this TrajReadOnly's coords
%timeit for i, frame in enumerate(trajread): frame[:] = xyz[i]

print ("mdtraj")
%timeit m_traj.xyz[:] = xyz

FrameArray
1 loops, best of 3: 284 ms per loop
api.Trajectory: frame iteration
1 loops, best of 3: 529 ms per loop
api.Trajectory: numpy
1 loops, best of 3: 236 ms per loop
TrajReadOnly
1 loops, best of 3: 1.3 s per loop
mdtraj
1 loops, best of 3: 213 ms per loop


## calc_rmsd

In [13]:
# save reference
f0 = fa[0]
%timeit for frame in fa: frame.rmsd(f0) 
%timeit for frame in traj: frame.rmsd(f0) 
%timeit for frame in trajread: frame.rmsd(f0)

# we can use fast rmsd calculation in mdtraj for our Trajectory too. (make the rmsd calculation ~ 3 times faster)
# for this calculation, mdtraj only cares about `xyz` attribute, and all of Trajectories in pytraj do have `xyz`
%timeit md.rmsd(traj, traj, 0)

1 loops, best of 3: 1.09 s per loop
1 loops, best of 3: 1.58 s per loop
1 loops, best of 3: 2.1 s per loop
1 loops, best of 3: 386 ms per loop


## calc_radgyr

In [14]:
%timeit fa.calc_radgyr()
%timeit traj.calc_radgyr()
%timeit trajread.calc_radgyr()

# trajread is the slowest because it needs to load frames to memory

1 loops, best of 3: 292 ms per loop
1 loops, best of 3: 887 ms per loop
1 loops, best of 3: 1.23 s per loop
